In [ ]:
import numpy as np
import pandas as pd
from math import isnan
import re
import matplotlib.pyplot as plt
from ape_producer.geom_reader import create_geom_df
from ape_producer.parse_report import make_cov_objects
from ape_producer.sign_conventions import signConventions, square_coordinates
from scipy.optimize import curve_fit
import os
import glob
#import mplhep as hep
%load_ext autoreload
%autoreload 2
pd.set_option('display.max_rows', 500)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import glob
from importlib import import_module
from ape_producer.parse_report import make_cov_objects, square_array
import numpy as np
import pandas as pd
import re

In [ ]:
from src.assets import *

In [ ]:
def sel_wheel_station(df, wheel, station):
    return df[(df.wheel==wheel) &  (df.station==station)]

def select_sectors(df, wheel, station):
    return df[(df.wheel==wheel) & (df.station==station)]

In [ ]:
####
##create list of xmls and reports that we want
###

In [ ]:
isRealistic = False
if isRealistic:
    re_xmls = [
    'data/realistic_10_6_.+.xml'
    ]
    re_report = [
    "data/realistic_10_6_.+.py"
    ] 
    dt_df_target, csc_df_target = create_geom_df('data/mctruth_realistic_10_6_2017.xml')
    dt_df_target, csc_df_target = pd.DataFrame(),  pd.DataFrame()
else:
    re_xmls = [
    "data/SM_10_6_.+.xml",
    'data/design_10_6_.+.xml'
    ]
    re_report = [
    "data/SM_10_6_.+.py",
    'data/design_10_6_.+.py'
    ]
    dt_df_target, csc_df_target = pd.DataFrame(),  pd.DataFrame()
def match_re(re_list, string):
    for re_string in re_list:
        if bool(re.match(re_string, string)): return 1
    return 0
files = glob.glob('data/*')
xml_files = [file for file in files if match_re(re_xmls, file)]
report_files = [file for file in files if match_re(re_report, file)]

In [ ]:
report_files

In [ ]:
def filter_list(file_list):
    design_list = list(filter(lambda x: 'design' in x, file_list))
    design_list = list(filter(lambda x: not '_0.' in x, design_list))
    csc = list(filter(lambda x: 'csc' in x, design_list))
    dt = list(filter(lambda x: not 'csc' in x, design_list))
    return dt

In [ ]:
design_xml_files = filter_list(xml_files)
design_report_files = filter_list(report_files)

In [ ]:
####
## Now, we create the report.py dataframe
####

In [ ]:
#format files as modules
modules = [f.replace('/', '.').replace('.py','') for f in design_report_files]

In [ ]:
def get_report(module_name):
    return import_module(module_name).reports

In [ ]:
def make_summary_report_df(module_name, njobs):
    report_ex = get_report(module_name)
    dt_cov_df, csc_cov_df = make_cov_objects(report_ex, debug=False)
    dt_cov_df['name'] = module_name.split('.')[1].replace('_report', '')
    return dt_cov_df

In [ ]:
def number_from_name(name):
    return int(re.findall('report_(\d+)', name)[0])

In [ ]:
stats_df = pd.DataFrame()
for module in modules:
    print(module)
    df = make_summary_report_df(module, number_from_name(module))
    stats_df = stats_df.append(df)

In [ ]:
def break_out_error(row):
    return pd.Series({a+'_unc':e for a,e in zip(alignables, row.error)})

In [ ]:
uncertainties = stats_df.apply(break_out_error, axis=1)

In [ ]:
stats_df = pd.concat([stats_df,uncertainties], axis=1)

In [ ]:
####
## Make xml df
####

In [ ]:
dt_df_combined = pd.DataFrame()
for xml_file in design_xml_files:
    xml_name = xml_file.split('/')[1].replace('.xml','')
    dt_df, csc_df = create_geom_df(xml_file)
    dt_df['xml_name'] = xml_name
    dt_df_combined = dt_df_combined.append(dt_df)

In [ ]:
dt_df_combined

In [ ]:
def get_value_wheel_station_name(w,s,sec, n):
    return stats_df[(stats_df.wheel==w) &
                    (stats_df.station==s) & 
                    (stats_df.sector==sec) & 
                   (stats_df.name==n)].iloc[0]

In [ ]:
get_value_wheel_station_name(-2,1, 1, "design_10_6_25M_01_1")

In [ ]:
stat_df_mapped = dt_df_combined.apply(lambda x: get_value_wheel_station_name(x.wheel, x.station, x.sector, x.xml_name), axis=1)

In [ ]:
combined_df = pd.concat([dt_df_combined.reset_index(drop=True),stat_df_mapped.reset_index(drop=True)], axis=1)

In [ ]:
combined_df.to_csv('combined_xml_report.csv')

In [ ]:
combined_df

In [ ]:
plt.scatter(abs(combined_df.x), combined_df.x_unc)
plt.plot([0,1], [0,1])